In [1]:
import sys
%load_ext autoreload
%autoreload 2
sys.path.append('..')
sys.path.append('../keras_bert/')

import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from keras_bert import bert_model

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [2]:
%env CUDA_VISIBLE_DEVICES=1

NUM_LABELS = 358

params = {
    'num_labels': NUM_LABELS,
    'label_list': [str(label) for label in range(NUM_LABELS)],
    'output_dir': '../output',
    'cache_dir': '../model_cache',
    'bert_model': 'bert-base-multilingual-uncased',
    'max_seq_length': 128,
    'train_batch_size': 32,
    'eval_batch_size': 8,
    'learning_rate': 2e-5,
    'warmup_proportion': 0.1,
    'num_train_epochs': 2,
    'seed': 1331,
    'device': torch.device(
        'cuda' if torch.cuda.is_available()
        else 'cpu')
}

params['lower_case'] = (params['bert_model'][-7:] == 'uncased')

env: CUDA_VISIBLE_DEVICES=1


## Preparing data

In [ ]:
DATA_DIR = '../datasets'

df_train = pd.read_csv(
    os.path.join(DATA_DIR, 'data/train.csv'),
    encoding='utf-8',
)
df_valid = pd.read_csv(
    os.path.join(DATA_DIR, 'data/dev.csv'),
    encoding='utf-8',
)
df_test = pd.read_csv(
    os.path.join(DATA_DIR, 'data/test.csv'),
    encoding='utf-8',
)

X_train, y_train = (
    df_train['text'].values,
    df_train['label_index'].values
)
X_valid, y_valid = (
    df_valid['text'].values,
    df_valid['label_index'].values
)
X_test, y_test = (
    df_test['text'].values,
    df_test['label_index'].values
)

## Training BERT model

In [4]:
model = bert_model.BertTextClassificationModel(params)

Completed!


In [5]:
result = model.fit(
    X_train,
    y_train,
    batch_size=32,
    n_epochs=1,
    validation_data=(X_valid, y_valid),
    best_model_output='model_saved.pth'
)
result


Epoch: 1


Iteration: 100%|██████████| 3273/3273 [27:45<00:00,  1.84it/s]


***** Running evaluation *****


Predicting: 100%|██████████| 3273/3273 [02:30<00:00, 21.73it/s]


{'train_log_loss': 2.225291140011025, 'eval_log_loss': 1.7633123544695246, 'eval_accuracy': 0.5897181269574517}


{'train_log_loss': 2.225291140011025,
 'eval_log_loss': 1.7633123544695246,
 'eval_accuracy': 0.5897181269574517,
 'best_epoch': 1,
 'model_filepath': '../output/model_saved.pth'}

## Evaluating the model

In [6]:
from lib import metrics

In [7]:
model.load(result['model_filepath'])

test_preds = model.predict(X_test)
y_pred = np.array(
    [np.argmax(probs) for i, probs in enumerate(test_preds)]
)

Predicting: 100%|██████████| 4365/4365 [02:48<00:00, 26.76it/s]


In [ ]:
print('Accuracy:  {}'.format(metrics.accuracy(y_test, test_preds)))
print('F1-micro:  {}'.format(metrics.f1_micro(y_test, test_preds)))
print('F1-macro:  {}'.format(metrics.f1_macro(y_test, test_preds)))
print('AUC-micro: {}'.format(metrics.auc_micro(y_test, test_preds)))
print('AUC-macro: {}'.format(metrics.auc_macro(y_test, test_preds)))
print()
for label in range(params['num_labels']):
    print('Label:', label)
    print('precision:', metrics.precision(y_test, test_preds, label))
    print('recall:   ', metrics.recall(y_test, test_preds, label))
    print()